# Temel sınıflandırma: Giysi görüntülerini sınıflandırın

Bu çalışma dosyasında, spor ayakkabı ve gömlek gibi giysi görüntülerini sınıflandırmak için bir sinir ağı modeli eğitiyor. 

TensorFlow'da modeller oluşturmak ve eğitmek için üst düzey (high-level) bir API olan tf.keras'ı kullanır.

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Fashion MNIST veri kümesi yükleme

Bu çalışmada, 10 kategoride 70.000 gri tonlamalı görüntü içeren [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) veri kümesini kullanır. 

Görüntüler, burada görüldüğü gibi düşük çözünürlükte (28 x 28 piksel) tek tek giyim eşyalarını göstermektedir:

<img src="https://tensorflow.org/images/fashion-mnist-sprite.png" alt="Moda MNIST hareketli grafiği" width="600">

<b>Şekil 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST örnekleri</a> (Zalando tarafından, MIT Lisansı).<br/> &nbsp;

Fashion MNIST, genellikle bilgisayarlı görme için makine öğrenimi programlarının "Merhaba Dünyası" olarak kullanılan klasik [MNIST](http://yann.lecun.com/exdb/mnist/) veri kümesinin yerini alacak şekilde tasarlanmıştır. . MNIST veri kümesi, burada kullanacağınız giyim eşyalarının formatına benzer bir biçimde el yazısı rakamların (0, 1, 2, vb.) görüntülerini içerir.

Bu çalışma dosyası, Fashion MNIST'i çeşitlilik için kullanır ve çünkü bu, normal MNIST'ten biraz daha zorlayıcı bir problemdir. Her iki veri kümesi de nispeten küçüktür ve bir algoritmanın beklendiği gibi çalıştığını doğrulamak için kullanılır. Kodu test etmek ve hata ayıklamak için iyi başlangıç ​​noktalarıdır.

Burada, ağı eğitmek için 60.000 görüntü ve ağın görüntüleri sınıflandırmayı ne kadar doğru öğrendiğini değerlendirmek için 10.000 görüntü kullanılır. Fashion MNIST'e doğrudan TensorFlow'dan erişebilirsiniz. Doğrudan TensorFlow'dan [Fashion MNIST verilerini yükleyebilirsiniz.](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data)

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Veri kümesi yüklendiğinde dört NumPy dizisi döndürür:

* "train_images" ve "train_labels" dizileri, *eğitim kümesidir*—modelin öğrenmek için kullandığı verilerdir.

* Model *test kümesi*, "test_images" ve "test_labels" dizilerine karşı test edilir.

Görüntüler, 0 ile 255 arasında değişen piksel değerlerine sahip 28x28 NumPy dizileridir. *etiketler(labels)*, 0 ile 9 arasında değişen bir tamsayı dizisidir. Bunlar, görüntünün temsil ettiği giysinin *sınıfına(class)* karşılık gelir:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Her görüntü tek bir etikete eşlenir. *Sınıf adları (class names)* veri kümesine dahil edilmediğinden, daha sonra görüntüleri çizerken kullanmak üzere bunları burada saklayın:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Verileri keşfedin

Modeli eğitmeden önce veri kümesinin biçimini keşfedelim. Aşağıdakiler, eğitim setinde her bir görüntünün 28 x 28 piksel olarak temsil edildiği 60.000 görüntü olduğunu göstermektedir:

**Yapılacaklar: eğitim resimlerinin boyutlarını yazdırın**

In [ ]:
# Yapılacaklar: eğitim resimlerinin boyutlarını yazdırın


Aynı şekilde eğitim setinde 60.000 adet etiket bulunmaktadır:

**Yapılacaklar: eğitim etiketlerinin sayısını yazdırın.**

In [ ]:
# Yapılacaklar: eğitim etiketlerinin sayısını yazdırın.


Her etiket 0 ile 9 arasında bir tamsayıdır:

In [ ]:
train_labels

Test kümesinde 10.000 adet görsel bulunmaktadır. Yine, her görüntü 28 x 28 piksel olarak temsil edilir:

In [ ]:
test_images.shape

Test kümesi 10.000 görüntü etiketi içerir:

In [ ]:
len(test_labels)

## Veri önişleme

Ağı eğitmeden önce veriler ön işleme tabi tutulmalıdır. Eğitim kümesindeki ilk görüntüyü incelerseniz piksel değerlerinin 0 ile 255 aralığında olduğunu göreceksiniz:

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Sinir ağı modeline beslemeden önce bu değerleri 0 ile 1 arasında ölçeklendirin. Bunu yapmak için değerleri 255'e bölün. *eğitim kümesi* ve *test kümesinin* aynı şekilde ön işleme tabi tutulması önemlidir:

**Yapılacaklar: eğitim ve test resimlerinin pixel değerlerini normalize edin**

In [ ]:
#Yapılacaklar: eğitim ve test resimlerinin pixel değerlerini normalize edin


Verilerin doğru formatta olduğunu ve ağı oluşturmaya ve eğitmeye hazır olduğunuzu doğrulamak için, *eğitim kümesinin* ilk 25 görüntüyü görüntüleyelim ve her görüntünün altında sınıf adını görüntüleyelim.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Modeli oluşturun

Sinir ağının oluşturulması, modelin katmanlarının yapılandırılmasını ve ardından modelin derlenmesini gerektirir.

### Katmanları ayarlayın

Bir sinir ağının temel yapı taşı [*katman(layer)*](https://www.tensorflow.org/api_docs/python/tf/keras/layers). 
Katmanlar, kendilerine beslenen verilerden temsilleri çıkarır. Umarım, bu temsiller eldeki sorun için anlamlıdır.

Derin öğrenmenin çoğu, basit katmanları birbirine zincirlemekten oluşur. "tf.keras.layers.Dense" gibi çoğu katman, eğitim sırasında öğrenilen parametrelere sahiptir.

**Yapılacaklar: CNN model mimarisini tanımlama**


-   **Flatten katmanı - giriş boyutu 28*28**

-   **Dense katmanı - 128 nöronlu ve relu aktivasyon fonksiyonlu**

-   **Çıkış (Dense) katmanı - sınıf sayısı kadar nörolu**



In [ ]:
# Yapılacaklar: CNN model mimarisini tanımlama




Bu ağdaki ilk katman olan `tf.keras.layers.Flatten`, görüntülerin formatını iki boyutlu bir diziden (28 x 28 piksel) tek boyutlu bir diziye (28 * 28 = 784 piksel) dönüştürür. Bu katman, görüntüdeki piksel sıralarını kaldırmak ve bunları hizalamak olarak düşünün. Bu katmanın öğrenilecek parametresi yoktur; yalnızca verileri yeniden biçimlendirir.

Pikseller düzleştirildikten sonra ağ, iki "tf.keras.layers.Dense" katmanından oluşan bir diziden oluşur. Bunlar yoğun(dense) şekilde bağlı veya tamamen bağlı sinir katmanlarıdır. İlk "Dense" katmanda 128 düğüm (veya nöron) bulunur. İkinci (ve son) katman, 10 uzunluğunda bir logits dizisi döndürür. Her düğüm, geçerli görüntünün 10 sınıftan birine ait olduğunu gösteren bir puan içerir.

### Modeli derleyin

Model eğitime hazır olmadan önce birkaç ayara daha ihtiyacı var. Bunlar, modelin [*compile*](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile) adımı sırasında eklenir:

* [*Yitim(Kayıp) Fonksiyonu (Loss function)*](https://www.tensorflow.org/api_docs/python/tf/keras/losses) —Bu, eğitim sırasında modelin ne kadar doğru olduğunu ölçer. Modeli doğru yöne "yönlendirmek" için bu işlevi en aza indirmek istiyorsunuz.

* [*Optimizer*](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) —Model, gördüğü verilere ve yitim fonksiyonuna göre bu şekilde güncellenir.

* [*Metrikler*](https://www.tensorflow.org/api_docs/python/tf/keras/metrics) —Eğitim ve test adımlarını izlemek için kullanılır. Aşağıdaki örnekte, doğru sınıflandırılmış görüntülerin kesri olan *doğruluk (accuracy)* kullanılmaktadır.

**Yapılacaklar:**

**- optimizasyon algoritması tanımlayınız**

**- metrics olarak "accuracy" tanımlayınız**

**- loss olarak çoklu sınıfa uygun tanımlayınız**

In [ ]:
# Modeli yukarıdaki açıklamadaki parametreler ile derleyin



## Modeli eğitin

Sinir ağı modelinin eğitimi aşağıdaki adımları gerektirir:

1. Eğitim verilerini modele besleyin. Bu örnekte, eğitim verileri "train_images" ve "train_labels" dizilerindedir.

2. Model, görüntüleri ve etiketleri ilişkilendirmeyi öğrenir.

3. Modelden bir test kümesi hakkında - bu örnekte "test_images" dizisi hakkında tahminler yapmasını istiyorsunuz.

4. Tahminlerin "test_labels" dizisindeki etiketlerle eşleştiğini doğrulayın.


### Modeli besleyin

Eğitime başlamak için [`model.fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) yöntemini çağırın.

**Yapılacaklar: Modeli 10 epoch eğitin.**

In [ ]:
# Yapılacaklar: Modeli 10 epoch eğitin.


Model eğitilirken, kayıp ve doğruluk metrikleri görüntülenir. 
Bu model, eğitim verilerinde yaklaşık 0.91 (veya %91) doğruluğa ulaşır.

### Doğruluğu değerlendirin

Ardından, modelin test veri kümesinde nasıl performans gösterdiğini karşılaştırın:

**Yapılacaklar: evaluate ile test görüntüleri için Test ACC hesaplayın**

In [ ]:
# Yapılacaklar: evaluate ile test görüntüleri için Test ACC hesaplayın


Test veri kümesinin doğruluğu, eğitim veri kümesindeki doğruluktan biraz daha az olduğu ortaya çıktı. 
Eğitim doğruluğu ve test doğruluğu arasındaki bu boşluk *aşırı öğrenme(overfitting)* temsil eder. 
Bir makine öğrenimi modeli, yeni, daha önce görülmemiş girdilerde eğitim verilerinde olduğundan daha kötü performans gösterdiğinde, aşırı öğrenme(overfitting) gerçekleşir. Aşırı öğrenen bir model, eğitim veri kümesindeki gürültüyü ve ayrıntıları ezberlemesi modelin yeni veriler üzerindeki performansını olumsuz yönde etkileyecektir.

Daha fazla bilgi için aşağıdakilere bakın:

*   [Demonstrate overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit#demonstrate_overfitting)
*   [Strategies to prevent overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit#strategies_to_prevent_overfitting)